In [1]:
import pandas as pd
from datetime import datetime
from main import *

In [2]:
dummy_data = {
    'shipid': ['shipA_202001', 'shipA_202001', 'shipB_202001', 'shipB_202001', 'shipC_202002', 'shipC_202002'],
    'date_time_utc': [
        datetime(2020, 1, 1, 8, 0),
        datetime(2020, 1, 31, 20, 0),
        datetime(2020, 1, 1, 9, 30),
        datetime(2020, 1, 31, 18, 0),
        datetime(2020, 2, 1, 7, 0),
        datetime(2020, 2, 28, 19, 0)
    ],
    'lat': [60.0, 60.5, 61.0, 61.5, 60.7, 61.2],
    'lon': [5.0, 5.2, 6.0, 6.3, 5.1, 5.5],
    'astd_cat': ['container ships', 'container ships', 'bulk carriers', 'bulk carriers', 'container ships', 'container ships'],
    'flagname': ['Norway', 'norway', 'Usa', 'usa', 'norway', 'Norway'],
    'sizegroup_gt': [5000, 5000, 7000, 7000, 5000, 5000],
    'iceclass': ['1A', '1A', '1C', '1C', '1A', '1A'],
}
dummy_df = pd.DataFrame(dummy_data)

In [3]:
dummy_df = preprocess_attributes(dummy_df)
dummy_df.head(10)

,shipid,date_time_utc,lat,lon,astd_cat,flagname,sizegroup_gt,iceclass
0,shipA_202001,2020-01-01 08:00:00,60.0,5.0,container ships,norway,5000,1a
1,shipA_202001,2020-01-31 20:00:00,60.5,5.2,container ships,norway,5000,1a
2,shipB_202001,2020-01-01 09:30:00,61.0,6.0,bulk carriers,usa,7000,1c
3,shipB_202001,2020-01-31 18:00:00,61.5,6.3,bulk carriers,usa,7000,1c
4,shipC_202002,2020-02-01 07:00:00,60.7,5.1,container ships,norway,5000,1a
5,shipC_202002,2020-02-28 19:00:00,61.2,5.5,container ships,norway,5000,1a


In [4]:
# Create buckets
dummy_df = bucket_creation(dummy_df)
dummy_df.head(10)

,shipid,date_time_utc,lat,lon,astd_cat,flagname,sizegroup_gt,iceclass,bucket
0,shipA_202001,2020-01-01 08:00:00,60.0,5.0,container ships,norway,5000,1a,container ships_norway_1a_5000
1,shipA_202001,2020-01-31 20:00:00,60.5,5.2,container ships,norway,5000,1a,container ships_norway_1a_5000
2,shipB_202001,2020-01-01 09:30:00,61.0,6.0,bulk carriers,usa,7000,1c,bulk carriers_usa_1c_7000
3,shipB_202001,2020-01-31 18:00:00,61.5,6.3,bulk carriers,usa,7000,1c,bulk carriers_usa_1c_7000
4,shipC_202002,2020-02-01 07:00:00,60.7,5.1,container ships,norway,5000,1a,container ships_norway_1a_5000
5,shipC_202002,2020-02-28 19:00:00,61.2,5.5,container ships,norway,5000,1a,container ships_norway_1a_5000


In [5]:
# Find start and end points
df_starts, df_ends = find_endpoints(dummy_df)

In [6]:
df_ends.head()

,shipid,date_time_utc,lat,lon,astd_cat,flagname,sizegroup_gt,iceclass,bucket
0,shipA_202001,2020-01-31 20:00:00,60.5,5.2,container ships,norway,5000,1a,container ships_norway_1a_5000
1,shipB_202001,2020-01-31 18:00:00,61.5,6.3,bulk carriers,usa,7000,1c,bulk carriers_usa_1c_7000
2,shipC_202002,2020-02-28 19:00:00,61.2,5.5,container ships,norway,5000,1a,container ships_norway_1a_5000


In [7]:
df_starts.head()

,shipid,date_time_utc,lat,lon,astd_cat,flagname,sizegroup_gt,iceclass,bucket
0,shipA_202001,2020-01-01 08:00:00,60.0,5.0,container ships,norway,5000,1a,container ships_norway_1a_5000
1,shipB_202001,2020-01-01 09:30:00,61.0,6.0,bulk carriers,usa,7000,1c,bulk carriers_usa_1c_7000
2,shipC_202002,2020-02-01 07:00:00,60.7,5.1,container ships,norway,5000,1a,container ships_norway_1a_5000


In [ ]:
# 1. Pick a single end point to test
sample_endpoint = df_ends.iloc[0]

# 2. Run the function
candidates = get_candidate_matches(sample_endpoint, df_starts)

# 3. Look at the result
candidates.head()  # or candidates[['shipid', 'time_hr', 'distance_km', ...]]

,shipid,date_time_utc,lat,lon,astd_cat,flagname,sizegroup_gt,iceclass,bucket,time_hr,distance_km
2,shipC_202002,2020-02-01 07:00:00,60.7,5.1,container ships,norway,5000,1a,container ships_norway_1a_5000,11.0,22.899093


In [ ]:
# TESTING USING ACTUAL DATASET

In [2]:
df_real = pd.read_csv('/Users/willponczak/Desktop/QC2025/data/grouped-months/all_months_1day.csv', sep=',')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [3]:
df_real.head(1)

,shipid,date_time_utc,flagname,iceclass,astd_cat,sizegroup_gt,fuelquality,fuelcons,co,co2,...,blackwater,greywater,garbage,dist_nextpoint,sec_nextpoint,longitude,latitude,year,month,day
0,146,2019-01-31 20:44:10,Iceland,FS Ice Class 1C,Fishing vessels,< 1000 GT,0,0.000075,5.533000e-07,0.000237,...,0.000025,0.000188,0.0,0.185819,22,-23.703434,64.89592,2019,1,31


In [4]:
result = build_track_table(df_real)

Processing 2776664 records
Data sample after cleaning:
  Date range: 2019-01-31 00:00:00 to 2019-12-01 23:59:57
  Ship types: ['fishing vessels' 'passenger ships' 'general cargo ships'
 'other activities' 'ro-ro cargo ships' 'offshore supply ships'
 'crude oil tankers' 'bulk carriers' 'refrigerated cargo ships'
 'oil product tankers' 'other service offshore vessels' 'chemical tankers'
 'cruise ships' 'container ships' 'gas tankers']
  Unique ships: 6832
Creating segments for 6832 unique shipids
Created 6832 segments
Sample segment: fishing vessels|iceland|fs ice class 1c|< 1000 gt
Found 6832 segments across 12 time periods


/Users/willponczak/Desktop/QC2025/TrackBuilder/track_builder/main.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_candidates['match_score'] = calculate_match_score(valid_candidates, ship_type)
/Users/willponczak/Desktop/QC2025/TrackBuilder/track_builder/main.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_candidates['match_score'] = calculate_match_score(valid_candidates, ship_type)
/Users/willponczak/Desktop/QC2025/TrackBuilder/track_builder/main.py:175: SettingWithCopyWarning: 
A v

Created 5296 tracks


In [6]:
result.shape

(6832, 3)

In [2]:
# ASTD Dataset after original conditioning 

df = pd.read_csv('/Users/willponczak/Desktop/QC2025/data/grouped-months/all_months_1day.csv', sep=',')

In [3]:
detailed_results = get_top_candidates_with_scores(df, 403, '2019-07', 3)


Data sample after cleaning:
  Date range: 2019-01-31 00:00:00 to 2019-12-01 23:59:57
  Ship types: ['fishing vessels' 'passenger ships' 'general cargo ships'
 'other activities' 'ro-ro cargo ships' 'offshore supply ships'
 'crude oil tankers' 'bulk carriers' 'refrigerated cargo ships'
 'oil product tankers' 'other service offshore vessels' 'chemical tankers'
 'cruise ships' 'container ships' 'gas tankers']
  Unique ships: 6832
Creating segments for 6832 unique shipids
Created 6832 segments
Sample segment: fishing vessels|iceland|fs ice class 1c|< 1000 gt
Found 1 candidates for segment 403
Score breakdown: lower scores = better matches


In [8]:
detailed_results

,ranking,segment_id,month,match_score,distance_km,time_gap_hours,implied_speed,month_gap,distance_score,speed_score,time_score,month_score,position_score
0,1,626,2019-08,0.1997,0.0,0.1,0.02,1,0.0,0.9971,0.0014,0.0,0.0


In [4]:
unfiltered_results = get_top_candidates_with_scores_unfiltered(df, 403, '2019-07', top_n=3)


Data sample after cleaning:
  Date range: 2019-01-31 00:00:00 to 2019-12-01 23:59:57
  Ship types: ['fishing vessels' 'passenger ships' 'general cargo ships'
 'other activities' 'ro-ro cargo ships' 'offshore supply ships'
 'crude oil tankers' 'bulk carriers' 'refrigerated cargo ships'
 'oil product tankers' 'other service offshore vessels' 'chemical tankers'
 'cruise ships' 'container ships' 'gas tankers']
  Unique ships: 6832
Creating segments for 6832 unique shipids
Created 6832 segments
Sample segment: fishing vessels|iceland|fs ice class 1c|< 1000 gt
Found 26 total candidates for segment 403
Showing top 3 candidates (including impossible ones)
Score breakdown: lower scores = better matches


In [5]:
unfiltered_results

,ranking,segment_id,month,match_score,distance_km,time_gap_hours,implied_speed,month_gap,distance_score,speed_score,time_score,month_score,position_score
0,1,626,2019-08,0.1997,0.00,0.1,0.02,1,0.0000,0.9971,0.0014,0.0,0.0000
1,2,4500,2019-08,0.4826,106.44,0.1,1011.08,1,0.2129,2.0000,0.0015,0.0,0.1844
2,3,3528,2019-08,0.4961,118.03,0.1,1323.76,1,0.2361,2.0000,0.0012,0.0,0.2508
